TO DO:
### Find relative numbers for initial supplies, required equipment, treatment node capacities, and treatment node times
### Create treatment path (steps in treatment, capacities of each node, time to complete each step)
### Write models in code and solve
#### Write and Format report
   

In [16]:
#treatment path 
using JuMP, Clp, NamedArrays

#tasks are hospital arrival(S), nurse evaluation(NE), condition decision(CD), breathing support decision(BS)
tasks = [:S, :NE, :CD, :BS, :HL, :LL, :D]
arcs = [(:S,:NE), (:NE,:CD), (:CD,:D), (:CD, :BS), (:BS, :HL), (:BD, :LL), (:LL,:D) (:HL, :D)]

#time(min) nurses needed per patient for the tasks per day, last 2 are cumulative total for day 
task_time = Dict(zip(tasks, [0, 45, 20, 30, 60])) 
#last node should be only 30 min for level 2's, 60 min for level 3's and 4's

#matrix for patient demand of each level each day
day = [:1,:2,:3,:4,:5,:6,:7]
severity = [:Pat_1, :Pat_2, :Pat_3, :Pat_4]
length_stay = [1, 3, 5, 10] #length of stay for each level of severity

#amount of time nurses spend with each patient severity per day 
time_per_level_day=[65, 125, 155, 155]

demand = NamedArray([8.0   7.0  13.0   8.0   9.0   7.0   9.0
                    23.0  24.0  26.0  27.0  28.0  30.0  31.0
                    11.0  11.0  11.0  11.0  12.0  12.0  12.0
                    11.0  11.0  11.0  11.0  12.0  12.0  12.0],(severity,day), ("Severity","Day"))

i=1     
for i <= 7 
    total_time_nurses = time_per_level_day[1] * demand[i,1] + time_per_level_day[2] * demand[i,2] + time_per_level_day[3] * demand[i,3]+time_per_level_day[4] * demand[i,4]
    i=i+1
end

total_time_nurses




LoadError: syntax: invalid iteration specification

In [21]:
#MCNF PROBLEM

using JuMP, Clp, NamedArrays
#incidence matrix (rows are nodes, columns are arcs, entries rep arc enters(-1) leaves(1) each node)
A=[1 0 0 0 0 0 0 0 
    -1 1 0 0 0 0 0 0
    0 -1 1 1 0 0 0 0 
    0 0 -1 0 1 1 0 0
    0 0 0 0 -1 0 1 0
    0 0 0 0 0 -1 0 1
    0 0 0 -1 0 0 -1 -1]
# add a dummy arc from sink (5) to source (0)
A = [A [-1;0;0;0;0;0;1]]

# demand
demand = NamedArray([8.0   7.0  13.0   8.0   9.0   7.0   9.0
                    23.0  24.0  26.0  27.0  28.0  30.0  31.0
                    11.0  11.0  11.0  11.0  12.0  12.0  12.0
                    11.0  11.0  11.0  11.0  12.0  12.0  12.0],(severity,day), ("Severity","Day"))

# costs should be 0 on every arc except dummy
# -1 on dummy arc
c = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1]

# capacities on each arc. make dummy arc capacity "big enough"
#cap = [10, 8, 5, 5, 4, 10, 7, 6, 3, 10, 14, 100]

m = Model(Clp.Optimizer)

# variables representing how much flow we send on each arc
@variable(m, x[1:9] >= 0)

# constraints balance flow into and out of each node
@constraint(m, A*x .== demand)

# don't exceed arc capacity
#@constraint(m, x .<= cap)

# minimize the total cost (same as maximizing flow through dummy arc)
@objective(m, Min, sum(c[i]*x[i] for i in 1:9))

# solve this instance of the max flow problem
optimize!(m)

# print out the flow on each arc along with total flow
println("Max flow: ", -objective_value(m)) # need to take negative
println("Flow on each arc: ")
println("S-NE: ", value(x[1]))
println("NE-CD: ", value(x[2]))
println("CD-BS: ", value(x[3]))
println("CD-D: ", value(x[4]))
println("BS-HL: ", value(x[5]))
println("BS-LL: ", value(x[6]))
println("HL-D: ", value(x[7]))
println("LL-D: ", value(x[8]))
println("Dummy: ", value(x[9]))

MethodError: MethodError: no method matching broadcasted_type(::Base.Broadcast.ArrayStyle{NamedArray{Float64,2,Array{Float64,2},Tuple{OrderedCollections.OrderedDict{Symbol,Int64},OrderedCollections.OrderedDict{Int64,Int64}}}}, ::Base.HasShape{2}, ::Type{GenericAffExpr{Float64,VariableRef}})
Closest candidates are:
  broadcasted_type(!Matched::Base.Broadcast.DefaultArrayStyle{N}, ::Base.HasShape{N}, ::Type{Eltype}) where {N, Eltype} at C:\Users\macke\.julia\packages\MutableArithmetics\NuiNA\src\broadcast.jl:2
  broadcasted_type(!Matched::SparseArrays.HigherOrderFns.PromoteToSparse, ::Base.HasShape{2}, ::Type{Eltype}) where Eltype at C:\Users\macke\.julia\packages\MutableArithmetics\NuiNA\src\sparse_arrays.jl:230
  broadcasted_type(!Matched::Base.Broadcast.DefaultArrayStyle{N}, ::Base.HasShape{N}, !Matched::Type{Bool}) where N at C:\Users\macke\.julia\packages\MutableArithmetics\NuiNA\src\broadcast.jl:5
  ...

In [18]:
A

7×8 Array{Int64,2}:
  1   0   0   0   0   0   0   0
 -1   1   0   0   0   0   0   0
  0  -1   1   1   0   0   0   0
  0   0  -1   0   1   1   0   0
  0   0   0   0  -1   0   1   0
  0   0   0   0   0  -1   0   1
  0   0   0  -1   0   0  -1  -1

In [3]:
#LONGEST PATH METHOD

pred_list[1,2] = 1;  pred_list[2,3] = 1;  pred_list[2,4] = 1;  pred_list[3,5] = 1; pred_list[4,6] = 1; pred_list[5,6] = 1;
pred_list[4,7] = 1; pred_list[6,8] = 1; pred_list[4,9] = 1; pred_list[6,9] = 1; pred_list[4,10] = 1; pred_list[6,11] = 1;
pred_list[9,12] = 1; pred_list[7,13] = 1; pred_list[2,14] = 1; pred_list[4,15] = 1; pred_list[14,15] = 1; 
pred_list[8,16] = 1; pred_list[11,16] = 1; pred_list[14,16] = 1; pred_list[12,17] = 1; pred_list[17,18] = 1; 
pred_list[10,20] = 1; pred_list[13,20] = 1; pred_list[15,20] = 1; pred_list[16,20] = 1; pred_list[18,20] = 1

### MODEL ###

m = Model(Clp.Optimizer)

@variable(m, x[T] >= 0) # variable for the time we begin each task

# create constraint for every pair of tasks (i,j) where task j is preceded by task i
@constraint(m,constr[i in T, j in T; pred_list[i,j] == 1], x[j] >= x[i] + duration[i])


# minimize the time we start task F (finish project)
@objective(m, Min, x[:F])

# solve this isntance of the longest path problem
optimize!(m)

# record the value of the variables
xsol = value.(x)

println("Earliest completion time: ", objective_value(m), " weeks")
for i in T
    println("Start task ", i , " in week ", xsol[i])
end

UndefVarError: UndefVarError: demand not defined

In [ ]:
#arc and node examples

nodes= [:s, :a, :b, :c, :d, :e, :f, :g, :h, :i, :j, :k, :l, :t]
arcs= [(:s,:a), (:s,:b), (:s,:c), (:a,:d), (:a,:e), (:a,:f), (:b,:d), (
:b,:e), (:b,:f), (:c,:d), (:c,:e), (:c,:f),
(:d,:g), (:d,:h), (:d,:i), (:e,:g), (:e,:h), (:e,:i), (:f,:g), (:f,:h),
(:f, :i), (:g,:j), (:g,:k), (:g,:l),
(:h,:j), (:h,:k), (:h,:l), (:i,:j), (:i,:k), (:i,:l), (:j,:t), (:k,:t),
(:l,:t)]
costs = Dict(zip(arcs,[750, 625, 1000, 625, 1500, 1700, 1425, 1000, 150
0, 2125, 2000, 500, 1000, 1625, 2450,
1800, 1125, 2250, 2500, 2125, 1250, 1500, 2050, 2325, 2300, 1250, 2125,
3000, 2250, 1125, 0, 0, 0]))
using JuMP, Clp, NamedArrays
m = Model(Clp.Optimizer)
@variable(m, x[arcs] >=0)
@objective(m, Min, sum(costs[a]*x[a] for a in arcs))
for i in nodes
    # if node is not source or sink
    if i != :s && i != :t
    # balance flow into and out of
    @constraint(m, sum(x[j] for j in arcs if j[2] == i) == sum(x[j]
        for j in arcs if j[1] == i))
    end
    @constraint(m,sum(x[j] for j in arcs if j[1] == i) == 1)
end
if i == :t # node is sink
@constraint(m,sum(x[j] for j in arcs if j[2] == i) == 1)
end
end
optimize!(m)
println("The minimized cost is \$", objective_value(m))
println("Build transportation routes as follows: ")
for a in arcs
println(" on arc", a, "send", value(x[a]))
end